#### Pratical AI

# PDF Processing

AI is restricted from using current data, and it can also hallucinate, so one of the ways we can test and also be sure of a specificly correct answer is by using a PDF Reader, in this example we will use arXiv to grab a bunch of papers, and question an AI

In this tutorial we will cover a few elements that comes mainly from the Aurelio SDK, more notably the PDF converter, and the chunking methods, we will also look at how to make sure everything is working properly.

In [1]:
arxiv_id = "2412.06634"  # Replace with the desired arXiv ID
pdf_url = f"https://arxiv.org/pdf/{arxiv_id}.pdf"

In [2]:
print(pdf_url)

https://arxiv.org/pdf/2412.06634.pdf


## Testing the URL

Firstly, it's super important to test the URL before we send it through the Aurelio SDK, as that costs money and we want to be certain what we have is a URL that works. We do this by importing some generic libraries, and we obtain the text and print the text out, checking for any errors, from this, we can gather that the URL is valid and that the PDF does have text we can interpret.

In [5]:
import requests
from io import BytesIO
import PyPDF2

# Download the PDF
response = requests.get(pdf_url)
response.raise_for_status()  # Check for errors

# Read the PDF content
pdf_file = BytesIO(response.content)
reader = PyPDF2.PdfReader(pdf_file)

# Extract text from all pages
text = ""
for page in reader.pages:
    text += page.extract_text()

# Display first 500 characters of the text
print(text[:500])

print(pdf_url)

  
 
                                                                                                                               Page 1 of 15 
 Observation of vortex-pair dance and oscillation 
 
Dadong Liu,1 Lai Chen,1 Li-Gang Wang 1,* 
 
1 School of Physics, Zhejiang University, Hangzhou 310058, China 
* Corresponding author. E-mail address: lgwang@zju.edu.cn (L.-G. Wang). 
 
Abstract 
Vortex dynamics, which encompass the motion, evolution, and propagation of vortices, 
elicit both fascinat
https://arxiv.org/pdf/2412.06634.pdf


## Using Aurelio SDK 

Firstly we need to make sure the system path is correct, to do this we append by ..

In [6]:
import sys

sys.path.append("..")

Next we will need an API key, this can be found by [logging into the Aurelio AI Platform website, from there go to settings, API keys, and you should be able to create a key there.](https://platform.aurelio.ai/settings/api-keys) Take note that the key will disapear so if you want to keep that key you should take a note of it somewhere!

In [7]:
import os
from getpass import getpass

os.environ["AURELIO_API_KEY"] = os.getenv("AURELIO_API_KEY") or getpass(
    "Enter Aurelio API Key: "
)

Now we want to initilize the Aurelio client.

In [9]:
from aurelio_sdk import AurelioClient

client = AurelioClient()

Here we can see all the functions we can use with the client!

In [10]:
print(dir(client))


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'api_key', 'base_url', 'chunk', 'embedding', 'extract_file', 'extract_url', 'get_document', 'headers', 'source', 'wait_for']


Now we want to test to make sure this API key is valid and working.

In [14]:
test_url = "https://arxiv.org/pdf/2408.15291"
try:
    response = client.extract_url(url=test_url, quality="low", chunk=True, wait=60, polling_interval=5)
    print("API key is valid.")
except Exception as e:
    print(f"API key is invalid: {e}")

API key is valid.


Here we can finally extract the text from the PDF, which does some additional stuff and gives us better results.

In [15]:
from aurelio_sdk import ExtractResponse
# extracting the pdf using Aurelio SDK
response_pdf_url: ExtractResponse = client.extract_url(
    url=pdf_url, quality="low", chunk=True, wait=60, polling_interval=5
)

response_pdf_url

ExtractResponse(status=<TaskStatus.completed: 'completed'>, usage=Usage(tokens=25464, pages=41, seconds=None), message=None, processing_options=ExtractProcessingOptions(chunk=True, quality=<ProcessingQuality.low: 'low'>), document=ResponseDocument(id='doc_8da1a2ea_5121_4457_9a11_a3ff14e88792', content="Observation of vortex-pair dance and oscillation\nDadong Liu,1 Lai Chen,1 Li-Gang Wang 1,*\n1 School of Physics, Zhejiang University, Hangzhou 310058, China * Corresponding author. E-mail address: lgwang@zju.edu.cn (L.-G. Wang).\nAbstract\nVortex dynamics, which encompass the motion, evolution, and propagation of vortices, elicit both fascination and challenges across various domains such as fluid dynamics, atmospheric science, and physics. This study focuses on fundamental dynamics of vortex-pair fields, specifically known as vortex-pair beams (VPBs) in optics. VPBs have gained increasing attention due to their unique properties, including vortex attraction and repulsion. Here, we explo

In [16]:
content = response_pdf_url.document.content

In [17]:
print(content[:2000])

Observation of vortex-pair dance and oscillation
Dadong Liu,1 Lai Chen,1 Li-Gang Wang 1,*
1 School of Physics, Zhejiang University, Hangzhou 310058, China * Corresponding author. E-mail address: lgwang@zju.edu.cn (L.-G. Wang).
Abstract
Vortex dynamics, which encompass the motion, evolution, and propagation of vortices, elicit both fascination and challenges across various domains such as fluid dynamics, atmospheric science, and physics. This study focuses on fundamental dynamics of vortex-pair fields, specifically known as vortex-pair beams (VPBs) in optics. VPBs have gained increasing attention due to their unique properties, including vortex attraction and repulsion. Here, we explore the dynamics of pure-phase VPBs (PPVPBs) and observe intriguing helical and intertwined behaviors of vortices, resembling a vortex- pair dance. We uncover the oscillation property of the intervortex distance for PPVPBs in free space. The observed dancing and oscillation phenomena are intricately tied to 

## Regex Chunking

Now we want to chunk the data, so we can process this is reasonable sizes, using the SDK we can chunk in regex, and pass our data into this and we will have everything we need!

In [522]:
from aurelio_sdk import ChunkingOptions, ChunkResponse

# All options are optional
chunking_options = ChunkingOptions(
    chunker_type="regex", delimiters=[], max_chunk_length=400
)

response_regex: ChunkResponse = client.chunk(
    content=content, processing_options=chunking_options
)

Useful to note that we can control the amount of chunks, and later on this becomes quite apparent, because Hugging Face can only process around 130 requests for new users, and if we have more, this means we will have to wait around for Hugging Face to allow us to upload the entire data set.

In [523]:
import json

for chunk in response_regex.document.chunks[:3]:
    print(json.dumps(chunk.model_dump(), indent=2))
    print("-" * 100)

{
  "id": "chunk_6c4d0f75-1074-4758-87c3-423b5d5a3a19",
  "content": "Observation of vortex-pair dance and oscillation Dadong Liu,1 Lai Chen,1 Li-Gang Wang 1,* 1 School of Physics, Zhejiang University, Hangzhou 310058, China * Corresponding author. E-mail address: lgwang@zju.edu.cn (L.-G. Wang). Abstract Vortex dynamics, which encompass the motion, evolution, and propagation of vortices, elicit both fascination and challenges across various domains such as fluid dynamics, atmospheric science, and physics. This study focuses on fundamental dynamics of vortex-pair fields, specifically known as vortex-pair beams (VPBs) in optics. VPBs have gained increasing attention due to their unique properties, including vortex attraction and repulsion. Here, we explore the dynamics of pure-phase VPBs (PPVPBs) and observe intriguing helical and intertwined behaviors of vortices, resembling a vortex- pair dance. We uncover the oscillation property of the intervortex distance for PPVPBs in free space. T

## Semantic Chunking

We want to compare the previous Regex chunking to the Semantic chunking, but before we do this, we want to chunk using the semantic method.

In [524]:
from aurelio_sdk import ChunkingOptions, ChunkResponse

# All options are optional
chunking_options = ChunkingOptions(
    chunker_type="semantic", max_chunk_length=400, window_size=5
)

response_semantic: ChunkResponse = client.chunk(
    content=content, processing_options=chunking_options
)

In [525]:
import json

for chunk in response_semantic.document.chunks[:3]:
    print(json.dumps(chunk.model_dump(), indent=2))
    print("-" * 100)

{
  "id": "chunk_5a175100-e782-4eac-b2cf-b9bef936b477",
  "content": "Observation of vortex-pair dance and oscillation Dadong Liu,1 Lai Chen,1 Li-Gang Wang 1,* 1 School of Physics, Zhejiang University, Hangzhou 310058, China * Corresponding author. E-mail address: lgwang@zju.edu.cn (L.-G. Wang).",
  "chunk_index": 1,
  "num_tokens": 68,
  "metadata": {}
}
----------------------------------------------------------------------------------------------------
{
  "id": "chunk_e60ff820-1763-48bf-80a0-d04a15f34e91",
  "content": "Abstract Vortex dynamics, which encompass the motion, evolution, and propagation of vortices, elicit both fascination and challenges across various domains such as fluid dynamics, atmospheric science, and physics. This study focuses on fundamental dynamics of vortex-pair fields, specifically known as vortex-pair beams (VPBs) in optics. VPBs have gained increasing attention due to their unique properties, including vortex attraction and repulsion. Here, we explore the

## Chunking Comparison

Now we want to compare both methods, to do this we will visually settup a small HTML window, showing both versions of the chunking process, by colouring each chunk differently to see how well or not so well they managed to process this data.

In [526]:
from IPython.display import HTML


def display_chunks_side_by_side(semantic_chunks, regex_chunks):
    """Display concatenated texts with chunks in different background colors side by side on white background"""
    print(f"Semantic chunks: {len(semantic_chunks)}")
    print(f"Regex chunks: {len(regex_chunks)}")

    colors = [
    "#FF6B6B",  # Soft Red
    "#FFD93D",  # Bright Yellow
    "#6BCB77",  # Soft Green
    "#4D96FF",  # Light Blue
    "#FF9F1A",  # Warm Orange
]
    text_color = "#011627"

    semantic_html = ""
    for idx, chunk in enumerate(semantic_chunks):
        color = colors[idx % len(colors)]
        semantic_html += (
            f'<span style="background-color:{color};">{chunk.content}</span>'
        )

    regex_html = ""
    for idx, chunk in enumerate(regex_chunks):
        color = colors[idx % len(colors)]
        regex_html += f'<span style="background-color:{color};">{chunk.content}</span>'

    html = f"""
    <table style="width:100%; table-layout:fixed; background-color:{text_color};">
        <tr>
            <th style="width:50%; text-align:left; color: {text_color}; ">Semantic Chunking</th>
            <th style="width:50%; text-align:left; color: {text_color};">Regex Chunking</th>
        </tr>
        <tr>
            <td style="vertical-align: top; text-align:left; color: {text_color};">{semantic_html}</td>
            <td style="vertical-align: top; text-align:left; color: {text_color};">{regex_html}</td>
        </tr>
    </table>
    """

    display(HTML(html))

In [527]:
display_chunks_side_by_side(
    response_semantic.document.chunks, response_regex.document.chunks
)

Semantic chunks: 279
Regex chunks: 67


Semantic Chunking,Regex Chunking


## Saving Data to Hugging Face

Now we have all our data we want to store this so we can use it at a later date, to do this, we use Hugging Face, you will need to sign in to get a username and also a token, [this can be done here](https://huggingface.co/welcome).

In [528]:
from huggingface_hub import HfApi, HfFolder
import os

os.environ["HUGGING_FACE_TOKEN"] = os.getenv("HUGGING_FACE_TOKEN") or getpass(
    "Enter Hugging Face Token: "
)

HfFolder.save_token(os.environ["HUGGING_FACE_TOKEN"])

api = HfApi()

os.environ["HUGGING_FACE_USERNAME"] = os.getenv("HUGGING_FACE_USERNAME") or getpass(
    "Enter Hugging Face Username: "
)

In [529]:
os.environ["HUGGING_FACE_REPO"] = os.getenv("HUGGING_FACE_REPO") or getpass(
    "Enter Hugging Face Repository Name: "
)

try:
    api.create_repo(repo_id=f"{os.environ["HUGGING_FACE_USERNAME"]}/{os.environ["HUGGING_FACE_REPO"]}", repo_type="dataset")
    print(f"Repository '{os.environ["HUGGING_FACE_REPO"]}' created successfully.")
except Exception as e:
    if "already" in str(e):
        print(f"Repository '{os.environ["HUGGING_FACE_REPO"]}' already exists.")
    else:
        print(f"Error creating repository: {e}")

Repository 'Article-Collection' already exists.


### Getting Additional Params

So we have now got all the credentials, and created a repo, we want to save this data, however first we want to process it, we need a name, description, and some meta data like tags, and then we want to store all of this alongside the content within a json to make it accessable and readable, to get this data, we need to use an AI to look for a title, summarise the article, and create tags, all of which based of the content of the article.

First we need to use a OpenAI key, as we will be using OpenAI's gpt-4o-mini model to scrape and summarise data.

In [530]:
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

client = OpenAI()

Now we want to define a function for extracting the title, this should look for a sentence from the start of the article, and decide which line should be the title.

In [531]:
def extract_title(content):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that extracts article titles. Return only the title, nothing else."
                },
                {
                    "role": "user",
                    "content": f"Extract the title from this article. Look at the first few lines as titles are usually at the top:\n\n{content[:500]}"
                }
            ],
            max_tokens=50,
            temperature=0.3
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error extracting title: {str(e)}"

Now we want to create a slug, basically we don't want spaces and punctuation as this will be getting processed and this can cause errors down the line if not handled properly.

In [532]:
import string

def create_slug(title):
    # Remove punctuation and convert to lowercase
    title = title.translate(str.maketrans("", "", string.punctuation)).lower()
    # Replace spaces with hyphens
    slug = title.replace(" ", "-")
    return slug

Now we want to create a summary, this will go through the article and summarise the article itself.

In [533]:
def summarize_article(content):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that provides concise summaries of articles."
                },
                {
                    "role": "user",
                    "content": f"Please provide a concise summary of the following article:\n\n{content}"
                }
            ],
            max_tokens=500,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generating summary: {str(e)}"

Lastly we want some tags to go along with our meta-data so we will once again go over the article and pick out some useful tags to describe the article.

In [534]:
def extract_tags(content, title):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that extracts relevant tags from articles. Return only a Python list of strings, with 5-8 relevant tags."
                },
                {
                    "role": "user",
                    "content": f"Based on this title and content, generate relevant tags. Return only a Python list of strings.\n\nTitle: {title}\n\nContent: {content[:1000]}"
                }
            ],
            max_tokens=100,
            temperature=0.3
        )
        # Convert the string response to an actual list
        tags_string = response.choices[0].message.content.strip()
        # Remove any markdown formatting and evaluate the string as a Python list
        tags_string = tags_string.replace('```python', '').replace('```', '')
        tags = eval(tags_string)
        return tags
    except Exception as e:
        return [f"Error extracting tags: {str(e)}"]

In [535]:
import logging
# Suppress httpx logging messages
logging.getLogger("httpx").setLevel(logging.WARNING)

In [536]:
# Extract and print the title
title = extract_title(response_pdf_url.document.content)
print("\nArticle Title:")
print("~" * 50)
print(title)
print("~" * 50)

filename = create_slug(title)
print("\nArticle File Name:")
print("~" * 50)
print(filename)
print("~" * 50)

summary = summarize_article(response_pdf_url.document.content)
print("\nArticle Summary:")
print("~" * 50)
print(summary)
print("~" * 50)

tags = extract_tags(response_pdf_url.document.content, title)
print("\nArticle Tags:")
print("~" * 50)
print(tags)
print("~" * 50)


Article Title:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Observation of vortex-pair dance and oscillation
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Article File Name:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
observation-of-vortexpair-dance-and-oscillation
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Article Summary:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The article "Observation of vortex-pair dance and oscillation" by Dadong Liu, Lai Chen, and Li-Gang Wang explores the dynamics of vortex-pair beams (VPBs) in the context of optical vortices. The study investigates pure-phase vortex-pair beams (PPVPBs), revealing unique behaviors such as helical and intertwined motions, described as a "vortex-pair dance." The authors observe oscillation in the distance between vortices, which is influenced by the initial distance between them, and find that this interaction can extend the survival range of opposite vortices, analogous to the Fujiwhara eff

### Uploading Data

Now we have all our pieces we can finally upload this dataset, we will create a combined data format holding all our variables in a json format, and then pass this in to be uploaded to Hugging Face.

In [537]:
import json

# Create a combined data structure
combined_data = [{
    "name": title,
    "id": arxiv_id,
    "content": response_pdf_url.document.content,
    "description": summary,
    "tags": ",".join(tags)
}]
data_filename = filename + ".json"
# Save the combined data to a single JSON file
with open(data_filename, "w", encoding='utf-8') as f:
    json.dump(combined_data, f, ensure_ascii=True, indent=2)

# Upload the single data file to Hugging Face
api.upload_file(
    path_or_fileobj=data_filename,
    path_in_repo=data_filename,
    repo_id=f"{os.environ["HUGGING_FACE_USERNAME"]}/{os.environ["HUGGING_FACE_REPO"]}",
    repo_type="dataset"
)

print(f"{filename} dataset uploaded successfully to {os.environ["HUGGING_FACE_REPO"]}!")

observation-of-vortexpair-dance-and-oscillation dataset uploaded successfully to Article-Collection!


## Downloading Data from Hugging Face

Now we have uploaded the data, we want that data back, so we will create a directory, and using that directory we will pull all the dataset back into our local folders.

In [538]:
print(data_filename)

observation-of-vortexpair-dance-and-oscillation.json


In [539]:
import os

# Create downloads directory if it doesn't exist
download_dir = "downloads"
os.makedirs(download_dir, exist_ok=True)

# Download the JSON file from Hugging Face
downloaded_file = api.hf_hub_download(
    repo_id=f"{os.environ["HUGGING_FACE_USERNAME"]}/{os.environ["HUGGING_FACE_REPO"]}",
    filename=data_filename,
    repo_type="dataset",
    local_dir=download_dir  # Specify the download directory
)

# Load and read the JSON content
with open(downloaded_file, 'r', encoding='utf-8') as f:
    downloaded_data = json.load(f)

print(f"Successfully downloaded {data_filename} to {download_dir}")

Successfully downloaded observation-of-vortexpair-dance-and-oscillation.json to downloads


## Finding Similar Articles

If you would like to gather a bigger pool of these articles for more in depth research, we can use the arXiv library to find us similar articles based of the name and tags.

In [543]:
import arxiv

def find_similar_articles(title, tags, max_results=5):
    tags_string = ' '.join(tags)
    search_query = f"{title} {tags_string}"
    
    client = arxiv.Client()
    search = arxiv.Search(
        query=search_query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )

    similar_articles = []
    for result in client.results(search):
        article = {
            "name": result.title,
            "id": result.entry_id.split('/')[-1],
            "content": result.summary,
            "description": result.summary,
            "tags": ",".join([cat.split('.')[-1] for cat in result.categories])
        }
        similar_articles.append(article)
    
    return similar_articles

In [544]:
similar_articles = find_similar_articles(title, tags)
print(f"Found {len(similar_articles)} similar articles:")
for article in similar_articles:
    print(f"\nTitle: {article['name']}")
    print(f"ID: {article['id']}")
    print(f"Tags: {article['tags']}")

2024-12-12 16:04:31 - arxiv - INFO - __init__.py:658 - __try_parse_feed() - Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=Observation+of+vortex-pair+dance+and+oscillation+vortex+dynamics+fluid+dynamics+optics+vortex-pair+beams+helical+behavior+oscillation+intervortex+distance+physics&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100
2024-12-12 16:04:33 - arxiv - INFO - __init__.py:584 - _results() - Got first page: 20 of 2623328 total results


Found 5 similar articles:

Title: Observation of vortex-pair dance and oscillation
ID: 2412.06634v1
Tags: optics,flu-dyn

Title: Hydrodynamic synchronization of autonomously oscillating optically trapped particles
ID: 1503.02555v1
Tags: flu-dyn,soft,optics

Title: Short range inter-vortex interaction and interacting dynamics of half-quantized vortices in two-component Bose-Einstein condensates
ID: 1510.00139v1
Tags: quant-gas,hep-ph

Title: Broad Leaves in Strong Flow
ID: 1310.5615v1
Tags: flu-dyn

Title: Alteration of helical vortex core without change in flow topology
ID: 1906.06391v1
Tags: flu-dyn


In [1]:
import re

get_id = re.compile(r'(?<=arxiv:)\d{4}.\d{5}')

In [2]:
get_id.findall(response_pdf_url.document.content)

NameError: name 'response_pdf_url' is not defined